# pyiron

https://github.com/BAMresearch/NFDI4IngScientificWorkflowRequirements

## Define workflow with pyiron_base

In [1]:
import os 

In [2]:
from workflow import (
    generate_mesh as _generate_mesh, 
    convert_to_xdmf as _convert_to_xdmf,
    poisson as _poisson,
    plot_over_line as _plot_over_line,
    substitute_macros as _substitute_macros,
    compile_paper as _compile_paper,
)

In [3]:
from pyiron_base import job

from python_workflow_definition.pyiron_base import write_workflow_json

In [4]:
generate_mesh = job(_generate_mesh)
convert_to_xdmf = job(_convert_to_xdmf, output_key_lst=["xdmf_file", "h5_file"])
poisson = job(_poisson, output_key_lst=["numdofs", "pvd_file", "vtu_file"])
plot_over_line = job(_plot_over_line)
substitute_macros = job(_substitute_macros)
compile_paper = job(_compile_paper)

In [5]:
domain_size = 2.0

In [6]:
source_directory = os.path.abspath(os.path.join(os.curdir, "source"))

In [7]:
gmsh_output_file = generate_mesh(
    domain_size=domain_size,
    source_directory=source_directory,
)

In [8]:
meshio_output_dict = convert_to_xdmf(
    gmsh_output_file=gmsh_output_file,
)

In [9]:
poisson_dict = poisson(
    meshio_output_xdmf=meshio_output_dict.output.xdmf_file, 
    meshio_output_h5=meshio_output_dict.output.h5_file,
    source_directory=source_directory,
)

In [10]:
pvbatch_output_file = plot_over_line(
    poisson_output_pvd_file=poisson_dict.output.pvd_file, 
    poisson_output_vtu_file=poisson_dict.output.vtu_file,
    source_directory=source_directory,
)

In [11]:
macros_tex_file = substitute_macros( 
    pvbatch_output_file=pvbatch_output_file, 
    ndofs=poisson_dict.output.numdofs, 
    domain_size=domain_size,
    source_directory=source_directory,
)

In [12]:
paper_output = compile_paper(
    macros_tex=macros_tex_file, 
    plot_file=pvbatch_output_file,
    source_directory=source_directory,
)

In [13]:
workflow_json_filename = "pyiron_base_nfdi.json"

In [14]:
write_workflow_json(delayed_object=paper_output, file_name=workflow_json_filename)

## Load Workflow with aiida

In [15]:
from aiida import load_profile

load_profile()

Profile<uuid='38168b192f654d6c9515c186d57ce4e1' name='pwd'>

In [16]:
from python_workflow_definition.aiida import load_workflow_json

In [17]:
wg = load_workflow_json(file_name=workflow_json_filename)
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…

In [18]:
wg.run()

05/14/2025 08:13:58 AM <465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [21|WorkGraphEngine|continue_workgraph]: tasks ready to run: generate_mesh6
05/14/2025 08:14:00 AM <465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [21|WorkGraphEngine|update_task_state]: Task: generate_mesh6, type: PyFunction, finished.
05/14/2025 08:14:00 AM <465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [21|WorkGraphEngine|continue_workgraph]: tasks ready to run: convert_to_xdmf5
05/14/2025 08:14:02 AM <465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [21|WorkGraphEngine|update_task_state]: Task: convert_to_xdmf5, type: PyFunction, finished.
05/14/2025 08:14:02 AM <465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [21|WorkGraphEngine|continue_workgraph]: tasks ready to run: poisson4
05/14/2025 08:14:10 AM <465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [21|WorkGraphEngin

## Load Workflow with jobflow

In [19]:
from python_workflow_definition.jobflow import load_workflow_json

In [20]:
from jobflow.managers.local import run_locally

In [21]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [22]:
result = run_locally(flow)
result

2025-05-14 08:15:14,382 INFO Started executing jobs locally
2025-05-14 08:15:14,545 INFO Starting job - generate_mesh (e5f18781-922f-446c-961b-3acdf00c5688)
2025-05-14 08:15:15,654 INFO Finished job - generate_mesh (e5f18781-922f-446c-961b-3acdf00c5688)
2025-05-14 08:15:15,655 INFO Starting job - convert_to_xdmf (2ee6c9fd-3a29-4f2a-bf4b-dd588ce58b79)
2025-05-14 08:15:16,962 INFO Finished job - convert_to_xdmf (2ee6c9fd-3a29-4f2a-bf4b-dd588ce58b79)
2025-05-14 08:15:16,963 INFO Starting job - poisson (ff73ebf0-2704-4c65-ab6d-a1a880634a1f)
2025-05-14 08:15:19,635 INFO Finished job - poisson (ff73ebf0-2704-4c65-ab6d-a1a880634a1f)
2025-05-14 08:15:19,636 INFO Starting job - plot_over_line (7225c5f3-ad66-425c-98b5-786d2a2da40d)
2025-05-14 08:15:21,057 INFO Finished job - plot_over_line (7225c5f3-ad66-425c-98b5-786d2a2da40d)
2025-05-14 08:15:21,059 INFO Starting job - substitute_macros (c1ae26db-a52a-491d-b5ea-52060b5fe0a7)
2025-05-14 08:15:21,885 INFO Finished job - substitute_macros (c1ae26

{'e5f18781-922f-446c-961b-3acdf00c5688': {1: Response(output='/home/jovyan/example_workflows/nfdi/preprocessing/square.msh', detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan/example_workflows/nfdi'))},
 '2ee6c9fd-3a29-4f2a-bf4b-dd588ce58b79': {1: Response(output={'xdmf_file': '/home/jovyan/example_workflows/nfdi/preprocessing/square.xdmf', 'h5_file': '/home/jovyan/example_workflows/nfdi/preprocessing/square.h5'}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan/example_workflows/nfdi'))},
 'ff73ebf0-2704-4c65-ab6d-a1a880634a1f': {1: Response(output={'numdofs': 357, 'pvd_file': '/home/jovyan/example_workflows/nfdi/processing/poisson.pvd', 'vtu_file': '/home/jovyan/example_workflows/nfdi/processing/poisson000000.vtu'}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_